In [1]:
from unipercept.nn import backbones
import tabulate
from IPython.display import display, HTML


def tabulate_available_models(_mod_name: str):
    avail = getattr(backbones, _mod_name).list_available()

    avail_split = [a.split("_", 1) for a in avail if "_" in a]
    avail_split += [(a, None) for a in avail if "_" not in a]

    avail_table = {}
    for cat, minor in avail_split:
        if minor is None:
            minor = "(base)"
        if cat not in avail_table:
            avail_table[cat] = []
        avail_table[cat].append(minor)

    for cat in avail_table.keys():
        avail_table[cat] = ", ".join(avail_table[cat]) if avail_table[cat] is not None else None

    avail_items = sorted(avail_table.items(), key=lambda x: x[0])
    avail_items = avail_items

    display(HTML(f"<h2 style='margin: auto'>{_mod_name}</h2>"))
    display(HTML(tabulate.tabulate(avail_items, tablefmt="html")))

## Testing Timm models

In [2]:
tabulate_available_models("timm")

bat
beit
beitv2
botnet26t
caformer
cait
coat
coatnet
coatnext
convformer
convit


In [3]:
tabulate_available_models("torchvision")

alexnet,(base)
convnext,"base, large, small, tiny"
deeplabv3,"mobilenet_v3_large, resnet101, resnet50"
densenet121,(base)
densenet161,(base)
densenet169,(base)
densenet201,(base)
efficientnet,"b0, b1, b2, b3, b4, b5, b6, b7, v2_l, v2_m, v2_s"
fasterrcnn,"mobilenet_v3_large_320_fpn, mobilenet_v3_large_fpn, resnet50_fpn, resnet50_fpn_v2"
fcn,"resnet101, resnet50"
fcos,resnet50_fpn


## Example backbone initialization

In [4]:
import unipercept as up
import torch
bb = up.nn.backbones.timm.TimmBackbone(name="convnext_small")

mock_inputs = torch.randn(2, 3, 512, 256)
mock_outputs = bb(mock_inputs)

print(mock_outputs)

/home/kstolle/.conda/envs/unipercept/lib/python3.11/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/home/kstolle/.conda/envs/unipercept/lib/python3.11/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/home/kstolle/.conda/envs/unipercept/lib/python3.11/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/home/kstolle/.conda/envs/unipercept/lib/python3.11/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,


TensorDict(
    fields={
        ext.1: Tensor(shape=torch.Size([2, 96, 128, 64]), device=cpu, dtype=torch.float32, is_shared=False),
        ext.2: Tensor(shape=torch.Size([2, 192, 64, 32]), device=cpu, dtype=torch.float32, is_shared=False),
        ext.3: Tensor(shape=torch.Size([2, 384, 32, 16]), device=cpu, dtype=torch.float32, is_shared=False),
        ext.4: Tensor(shape=torch.Size([2, 768, 16, 8]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([2]),
    device=cpu,
    is_shared=False)


## Feature Pyramid Networks (FPN)


In [5]:
from unipercept.nn import backbones
from pprint import pprint

for builder in (backbones.fpn.build_default_routing, backbones.fpn.build_pan_routing, backbones.fpn.build_quad_routing):
    print(builder.__name__)
    pprint(builder(1, 5))

build_default_routing


ValueError: 5 is not a valid WeightMethod

In [ ]:
from unipercept.nn import backbones
from pprint import pprint

bb = backbones.torchvision.TorchvisionBackbone("resnet50")

print("Feature info:")
pprint(bb.feature_info)

routing = backbones.fpn.build_default_routing(1, 5)

print("FPN routing:")
pprint(routing)

bb_fpn = backbones.fpn.FeaturePyramidBackbone(bb, out_channels=64, routing=routing)

print("FPN backbone feature info:")
pprint(bb_fpn.feature_info)

Feature info:
[BackboneFeatureInfo(channels=256, stride=4),
 BackboneFeatureInfo(channels=512, stride=8),
 BackboneFeatureInfo(channels=1024, stride=16),
 BackboneFeatureInfo(channels=2048, stride=32)]
FPN routing:
{'max_level': 5,
 'min_level': 1,
 'nodes': [{'in_offsets': [3, 4],
            'level': 4,
            'weight_method': <WeightMethod.FAST_ATTENTION: 'fastattn'>},
           {'in_offsets': [2, 5],
            'level': 3,
            'weight_method': <WeightMethod.FAST_ATTENTION: 'fastattn'>},
           {'in_offsets': [1, 6],
            'level': 2,
            'weight_method': <WeightMethod.FAST_ATTENTION: 'fastattn'>},
           {'in_offsets': [0, 7],
            'level': 1,
            'weight_method': <WeightMethod.FAST_ATTENTION: 'fastattn'>},
           {'in_offsets': [1, 7, 8],
            'level': 2,
            'weight_method': <WeightMethod.FAST_ATTENTION: 'fastattn'>},
           {'in_offsets': [2, 6, 9],
            'level': 3,
            'weight_method': <We